In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df_data = pd.read_csv("adult.csv")

In [3]:
df_data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
df_data.dtypes

age                 int64
workclass          object
fnlwgt              int64
education          object
educational-num     int64
marital-status     object
occupation         object
relationship       object
race               object
gender             object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
income             object
dtype: object

In [5]:
df_data["native-country"].value_counts().index[0]

' United-States'

In [6]:
df_data["income"].value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

In [7]:
df_data = df_data.drop(['fnlwgt', 'educational-num'], axis = 1) 

In [8]:
col_names = df_data.columns

In [9]:
for c in col_names: 
    df_data = df_data.replace("?", np.NaN) 
df_data = df_data.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [10]:
category_col =['workclass', 'education', 'marital-status', 'occupation', 'relationship', 
               'race', 'gender', 'native-country', 'income']  
labelEncoder = preprocessing.LabelEncoder() 
  
mapping_dict ={} 
for col in category_col: 
    df_data[col] = labelEncoder.fit_transform(df_data[col]) 
  
    le_name_mapping = dict(zip(labelEncoder.classes_, 
                        labelEncoder.transform(labelEncoder.classes_))) 
  
    mapping_dict[col]= le_name_mapping 
print(mapping_dict) 

{'marital-status': {' Married-spouse-absent': 3, ' Never-married': 4, ' Married-AF-spouse': 1, ' Married-civ-spouse': 2, ' Divorced': 0, ' Widowed': 6, ' Separated': 5}, 'income': {' <=50K': 0, ' >50K': 1}, 'native-country': {' Puerto-Rico': 33, ' Guatemala': 13, ' Hungary': 18, ' China': 3, ' Dominican-Republic': 6, ' England': 9, ' Canada': 2, ' France': 10, ' India': 19, ' Japan': 24, ' El-Salvador': 8, ' Outlying-US(Guam-USVI-etc)': 28, ' Honduras': 16, ' Columbia': 4, ' Ireland': 21, ' Vietnam': 40, ' Cambodia': 1, ' Iran': 20, ' Trinadad&Tobago': 38, ' Portugal': 32, ' United-States': 39, ' Hong': 17, ' Jamaica': 23, ' ?': 0, ' Philippines': 30, ' Nicaragua': 27, ' Mexico': 26, ' Poland': 31, ' Cuba': 5, ' Italy': 22, ' Haiti': 14, ' Yugoslavia': 41, ' Holand-Netherlands': 15, ' Germany': 11, ' Thailand': 37, ' Peru': 29, ' Laos': 25, ' Taiwan': 36, ' Scotland': 34, ' South': 35, ' Ecuador': 7, ' Greece': 12}, 'relationship': {' Husband': 0, ' Other-relative': 2, ' Not-in-family'

In [11]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
  
X = df_data.drop('income', axis = 1) 
Y = df_data['income']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100) 

In [12]:
import warnings
warnings.filterwarnings('ignore')

## Bagging

In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

lr_clf = LogisticRegression() 
dt_clf = DecisionTreeClassifier() 
rf_clf = RandomForestClassifier() 
et_clf = ExtraTreesClassifier() 
knn_clf = KNeighborsClassifier() 
ensembler = VotingClassifier(estimators=[('LogisticRegression', lr_clf), ('DecisionTreeClassifier', dt_clf),
                                        ('RandomForestClassifier', rf_clf), ('ExtraTreesClassifier', et_clf),
                                        ('KNeighborsClassifier', knn_clf)], voting = 'hard')

classifiers =[lr_clf, dt_clf, rf_clf, et_clf, knn_clf]
for clf in classifiers:
    clf_scores = cross_val_score(clf, X_train, y_train, cv = 10)
    bagging_clf = BaggingClassifier(clf, max_samples=0.4, max_features=10, random_state=0)
    bagging_clf_scores = cross_val_score(bagging_clf, X_train, y_train, cv = 10)
    print(clf.__class__.__name__, ":::: Mean:", clf_scores.mean(), ", Std Dev:", clf_scores.std())
    print("Bagging", clf.__class__.__name__, ":::: Mean:", bagging_clf_scores.mean(), ", Std Dev:", bagging_clf_scores.std(), "\n")

ensembler = VotingClassifier(estimators=[('LogisticRegression', lr_clf), ('DecisionTreeClassifier', dt_clf),
                                        ('RandomForestClassifier', rf_clf), ('ExtraTreesClassifier', et_clf),
                                        ('KNeighborsClassifier', knn_clf)], voting = 'hard')


ensembler_scores = cross_val_score(ensembler, X_train, y_train, cv = 10)
print(ensembler.__class__.__name__, ":::: Mean:", ensembler_scores.mean(), ", Std Dev:", clf_scores.std())

LogisticRegression :::: Mean: 0.8006753500688977 , Std Dev: 0.004969066102330903
Bagging LogisticRegression :::: Mean: 0.8037462760675274 , Std Dev: 0.005739327540419379 

DecisionTreeClassifier :::: Mean: 0.8132235206269293 , Std Dev: 0.0059799546800177955
Bagging DecisionTreeClassifier :::: Mean: 0.849639923635328 , Std Dev: 0.0046034229502244905 

RandomForestClassifier :::: Mean: 0.8428395418119674 , Std Dev: 0.003365797368294014
Bagging RandomForestClassifier :::: Mean: 0.8571426949339122 , Std Dev: 0.0043497027492810135 

ExtraTreesClassifier :::: Mean: 0.8357312764139395 , Std Dev: 0.006725800884171518
Bagging ExtraTreesClassifier :::: Mean: 0.8507809096017798 , Std Dev: 0.0048621773984696935 

KNeighborsClassifier :::: Mean: 0.8238853221249702 , Std Dev: 0.006423083088668752
Bagging KNeighborsClassifier :::: Mean: 0.8396364017767104 , Std Dev: 0.00599320955270458 

VotingClassifier :::: Mean: 0.8454716211326913 , Std Dev: 0.006423083088668752


## Boosting

In [26]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

ab_clf = AdaBoostClassifier() 
gb_clf = GradientBoostingClassifier() 
xgb_clf = XGBClassifier() 

ensembler = VotingClassifier(estimators=[('AdaBoostClassifier', ab_clf), ('GradientBoostingClassifier', gb_clf),
                                        ('XGBClassifier', xgb_clf)], voting = 'hard')

classifiers =[ab_clf, gb_clf, xgb_clf, ensembler]
for clf in classifiers:
    clf_scores = cross_val_score(clf, X_train, y_train, cv = 10)
    print(clf.__class__.__name__, ":::: Mean:", clf_scores.mean(), ", Std Dev:", clf_scores.std())

AdaBoostClassifier :::: Mean: 0.8604337082284473 , Std Dev: 0.0032409094349287403
GradientBoostingClassifier :::: Mean: 0.8644262257222698 , Std Dev: 0.0032315430892614675
XGBClassifier :::: Mean: 0.8641189579917322 , Std Dev: 0.004561102596800773
VotingClassifier :::: Mean: 0.864645581703271 , Std Dev: 0.0032985215353102735


## Stacking

In [30]:
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier

def get_stacking():
    level0 = [('lr', LogisticRegression()), ('knn', KNeighborsClassifier()), ('cart', DecisionTreeClassifier()),
              ('svm', SVC()), ('bayes', GaussianNB())]
    level1 = LogisticRegression()
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model
 
def get_models():
    models = dict()
    models['lr'] = LogisticRegression()
    models['knn'] = KNeighborsClassifier()
    models['cart'] = DecisionTreeClassifier()
    models['svm'] = SVC()
    models['bayes'] = GaussianNB()
    models['stacking'] = get_stacking()
    return models
 
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
    return scores
 
models = get_models()
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X_train, y_train)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

ImportError: cannot import name 'StackingClassifier'